# Configurando o ambiente

Atualmente, o LangChain ainda não contém todos os features disponíveis no CassIO, estas PRs estão sendo adicionadas aos poucos.

Instale as dependencias baseadas no arquivo requirements.txt

In [ ]:
pip install -r requirements.txt

# Defina as variáveis de ambiente

Após criar sua conta no DataStax Astra (astra.datastax.com), faça download do Bundle de conexão, e gere um token de acesso.

Todo este processo está descrito aqui: https://cassio.org/start_here/#create-the-database

Com estas informações abra o arquivo ".env_sample", preencha as variáveis e salve como ".env"

In [1]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

# Conexão com o Astra

Este repositório contém o arquivo cqlsession.py que é responsável por criar a conexão com o DataStax Astra.

Este arquivo é utilizado assim:

In [62]:
# Inicializando a sessão com o Astra/Cassandra
# A criação da sessão é realizada pelo arquivo cqlsession.py
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

## Criando dados para personalizar o prompt

A ideia é personalizar o prompt com informações gravadas no Cassandra, então vamos criar uma tabela que armazena o idioma e linguagem de programação para um determinado usuário.

In [14]:
# Criação de uma tabela para armazenar os dados personalizados.
from cassandra.query import SimpleStatement
session.execute(SimpleStatement("CREATE TABLE IF NOT EXISTS demo.my_user (user_id TEXT PRIMARY KEY, language TEXT, programming_language TEXT)"))

In [16]:
# Inserindo dados que serão usados na personalização dos prompts
session.execute("INSERT INTO demo.my_user (user_id, language, programming_language ) VALUES ('samuel-br','portugues','javascript')")
session.execute("INSERT INTO demo.my_user (user_id, language, programming_language ) VALUES ('samuel-en','ingles','python')")
session.execute("INSERT INTO demo.my_user (user_id, language, programming_language ) VALUES ('samuel-fr','francês','c++')")
session.execute("INSERT INTO demo.my_user (user_id, language, programming_language ) VALUES ('samuel-es','espanhol','java')")


In [20]:
# Consultando o resultado
data = session.execute("SELECT * FROM demo.my_user")
for row in data:
    print (row.user_id, row.language, row.programming_language)

samuel-fr francês c++
samuel-br portugues javascript
samuel-es espanhol java
samuel-en ingles python


# Criando o primeiro prompt

Toda interação com o LLM é feita através de prompts. Estes prompts são ainda mais poderosos quando utilizam dados personalizados.

O que faremos neste exemplo é utilizar dados que estão gravados no Astra para personalizar o prompt.

In [27]:
# Definição do prompt
ctemplate0 = """Please answer a question from a user in {user_language}.
Keep in mind that the preferred programing language of the user is {user_programming_language}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

In [22]:
from langchain.prompts import createCassandraPromptTemplate

In [28]:
# Criação do prompt dinâmico
# O nome de alguma das variáveis de entrada devem corresponder à chave primaria da tabela. Neste caso, user_id é a PK, então temos que usar esta variável.
cassPrompt = createCassandraPromptTemplate(
    session=session,
    keyspace=keyspace,
    template=ctemplate0,
    input_variables=['user_id', 'user_question'],
    field_mapper={
        'user_language': ('my_user', 'language'),
        'user_programming_language': ('my_user', 'programming_language'),
    },
)

Apenas para entendermos o que aconteceu, aqui vemos qual vai ser o prompt gerado dinamicamente

In [33]:
# Consultando o resultado do prompt dinâmico
print(cassPrompt.format(user_id='samuel-br', 
                        user_question='Which is the function that returns the size of an array?'))

Please answer a question from a user in portugues.
Keep in mind that the preferred programing language of the user is javascript.

USER'S QUESTION: Which is the function that returns the size of an array?
YOUR ANSWER:



In [34]:
# Importando o modelo LLM. Neste caso estamos usando a OpenAI, mas você pode usar qualquer um
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")

In [40]:
# Resposta em português
res = llm(cassPrompt.format(user_id='samuel-br', 
                        user_question='What is the function that returns the array size? can you explain how to use it?'))
print(res)

A função que retorna o tamanho de um array em JavaScript é o .length. Para usá-lo, você pode acessar o tamanho de um array atribuindo o .length ao array desejado, como no exemplo abaixo: 

var arr = ["uma", "duas", "três"];
var size = arr.length; // retorna 3


In [39]:
res = llm(cassPrompt.format(user_id='samuel-es', 
                        user_question='How can I create a class? Can you give me an example?'))
print(res)

Para crear una clase en Java, primero debe definir sus atributos y métodos. Por ejemplo, aquí hay una clase sencilla para una persona que contiene los atributos de nombre y edad: 
public class Persona {
  String nombre;
  int edad;
}


In [41]:
res =llm(cassPrompt.format(user_id='samuel-fr', 
                        user_question='Can you give me an example of how to create a vector?'))
print(res)

Voici un exemple pour créer un vecteur en C++ : 
std::vector<int> monVecteur; // créer un vecteur vide 
monVecteur.push_back(10); // ajouter 10 à la fin du vecteur 
monVecteur.push_back(20); // ajouter 20 à la fin du vecteur


# System Message

O jeito mais correto de informar no prompt o comportamento do modelo usando uma System Message.

Por isso, faremos a mesma personalização, só que desta vez na mensagem do sistema.

In [53]:
# Definição do template
systemTemplate = """Please answer a question from a user in {user_language}.
Keep in mind that the preferred programing language of the user is {user_programming_language}.
"""

In [54]:
# Buscando os valores da tabela
cassSystemPrompt = createCassandraPromptTemplate(
    session=session,
    keyspace=keyspace,
    template=systemTemplate,
    input_variables=['user_id', 'user_question'],
    field_mapper={
        'user_language': ('my_user', 'language'),
        'user_programming_language': ('my_user', 'programming_language'),
    },
)

In [55]:
# Criação da mensagem de sistema
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
systemMessagePrompt = SystemMessagePromptTemplate(prompt=cassSystemPrompt)

In [56]:
# Criação da mensagem humana, que agora só possui o texto enviado pelo usuário

humanTemplate = "{text}"
humanMessagePrompt = HumanMessagePromptTemplate.from_template(humanTemplate)


In [57]:
# Criação do prompt completo
cassChatPrompt = ChatPromptTemplate.from_messages(
    [systemMessagePrompt, humanMessagePrompt]
)

In [58]:
# Exibindo o prompt gerado utilizando as variáveis e enriquecido com dados da tabela
print(cassChatPrompt.format_prompt(
    user_id='samuel-br', 
    text='What is the function that returns the array size? can you explain how to use it?'
).to_string())

System: Please answer a question from a user in portugues.
Keep in mind that the preferred programing language of the user is javascript.

Human: What is the function that returns the array size? can you explain how to use it?


In [60]:
# Criação da chamada de chat da OpenAI, especificando o modelo
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [61]:
# Execução do modelo, passando as variaveis e a questão do usuário
res = llm(cassChatPrompt.format_prompt(
    user_id='samuel-br', 
    text='What is the function that returns the array size? can you explain how to use it?'
).to_string())
print(res, end="\n")



System: A função que retorna o tamanho de um array é o .length. Para usá-lo, basta inserir o nome do array seguido de um ponto e o nome da função: array.length. Por exemplo, se você tiver um array chamado "meuArray", a função para determinar o seu tamanho seria meuArray.length.
